# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БВТ2001 Ильинская Вера

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Алгоритм Флойда-Уоршелла

In [ ]:
#построение специальной матрицы кратчайших путей для всех вершин графа

###Алгоритм Дейкстры

In [15]:
#алгоритм нахождения кратчайшей путей от вершины 1 до всех остальных вершин

import math

try:
  #считываем текстовый файл
  with open("/content/drive/MyDrive/CIAOD and VVIT/Lab6Files/DijkstraAlgo.txt", "r", encoding = "utf-8") as fileDij:
    matrix = [[int(num) for num in line.split(',')] for line in fileDij] #загружаем матрицу дорог графа
    matrix = ((0,3,1,3,0,0), #с загрузкой матрицы пока работает некорректно 
              (3,0,4,0,0,0),
              (1,4,0,0,7,5),
              (3,0,0,0,0,2),
              (0,0,7,0,0,4),
              (0,0,5,2,4,0))
    N = len(matrix) #число вершин в нашем графе
    T = [math.inf] * N #последняя строка таблицы Дейкстры (inf - бесконечность)
    startV = 0 #стартовая вершина
    viewedV = {startV} #множество просмотренных вершин (в зависимости от старо-й вершины)
    T[startV] = 0 #нулевой вес старостовой вершины должен быть равен 0!

    #метод перебора всех связанных вершин
    #просматриваем все веса других, связанных с V вершин, и возвращаем вершину с мин.значением
    def getLinkWithV(startV, matrix):
      for i, weight in enumerate(matrix[startV]): #берём веса в графе 
        if (weight > 0):
          yield i

    #метод нахождения в таблицы вершины с мин.весом
    def findMinWei(T, viewedV):
      amin = -1 #начальный минимум, начальное состояние
      m = max(T) #находим макс.значение последней строки
      for i, t in enumerate(T):
        if (t < m and i not in viewedV): #находим мин.значение вершин, которые не были ещё рассмотрены
            m = t
            amin = i
      return amin #возвращаем вершину с мин.значением

    #заполяем T-табличку мин.весами
    while (startV != -1): #цикл для прохождению по вершинам (если -1, то по всем вершинам прошли)
      for j in getLinkWithV(startV, matrix): #перебераем все связанные вершины с вершиной startV
        if (j not in viewedV): #если рассматриваемая строка таблицы не была просмотрена
          wei = T[startV] + matrix[startV][j] #получаем путь, что прошли до минимальный вершины
          if (wei < T[j]): #если вес меньше последней строки нашей таблицы
            T[j] = wei #присваиваем минимальное значение, то есть меняем мин.строку
      #должны найти мин.аргумент между последней строчкой таблицы и множеством, что уже было просмотрено
      startV = findMinWei(T, viewedV) #находим мин.значение вершины
      if (startV > 0):
        viewedV.add(startV)
    print("Длины кратчайших путей от вершины 1 до всех остальных вершин соотвественно: ")
    print(T)

except FileNotFoundError:
  print("Файл не был найден") 

Длины кратчайших путей от вершины 1 до всех остальных вершин соотвественно: 
[0, 3, 1, 3, 8, 5]


### Вывод